In [51]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional
from keras.optimizers import adam_v2
# from tensorflow.keras.optimizers import Adam
from numpy.random import seed
from utils import *
from model import *


# GPU
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    tf.config.set_visible_devices([gpus[0]], "GPU")

seed(1)
tf.random.set_seed(1)

n_timestamp = 10
n_epochs = 50
# ====================================
#      model type：
#            1. single-layer LSTM
#            2. multi-layer LSTM
#            3. bidirectional LSTM
# ====================================


yuan_data = pd.read_csv('601988.SH.csv')  
yuan_data.index = pd.to_datetime(yuan_data['trade_date'], format='%Y%m%d') 
yuan_data = yuan_data.loc[:, ['open', 'high', 'low', 'close', 'amount']]
yuan_data

,open,high,low,close,amount
trade_date,,,,,
2007-01-04,5.69,5.97,5.37,5.63,4224852.614
2007-01-05,5.30,5.34,5.07,5.07,4050377.943
2007-01-08,4.87,5.14,4.83,5.08,2763751.983
2007-01-09,5.06,5.19,4.95,5.18,2198327.249
2007-01-10,5.25,5.29,5.05,5.10,1915221.182
...,...,...,...,...,...
2022-03-11,3.08,3.11,3.06,3.10,550569.534
2022-03-14,3.08,3.12,3.07,3.09,489277.432
2022-03-15,3.09,3.09,3.02,3.03,886628.898


In [52]:

data = pd.read_csv('ARIMA_residuals1.csv')
data.index = pd.to_datetime(data['trade_date'])
data = data.drop('trade_date', axis=1)
# data = pd.merge(data, yuan_data, on='trade_date') 


In [65]:


Lt = pd.read_csv('ARIMA.csv')
idx = 3500
training_set = data.iloc[1:idx, :]
test_set = data.iloc[idx:, :]
yuan_training_set = yuan_data.iloc[1:idx, :]
yuan_test_set = yuan_data.iloc[idx:, :]


In [ ]:
history_yuan = [x for x in yuan_training_set['close']]
predictions_yuan = list()
for t in range(len(yuan_test_set)):
    model1 = sm.tsa.ARIMA(history_yuan, order=(2, 1, 0))
    model_fit = model1.fit()
    yhat = model_fit.forecast()
    predictions_yuan.append(yhat[0])
    obs = yuan_test_set.iloc[t, 3]
    history_yuan.append(obs)

In [42]:
history = [x for x in training_set.iloc[:,0]]
predictions = list()
for t in range(len(test_set)):
    model1 = sm.tsa.ARIMA(history, order=(2, 1, 0))
    model_fit = model1.fit()
    yhat = model_fit.forecast()
    predictions.append(yhat[0])
    obs = test_set.iloc[t, 0]
    history.append(obs)


In [81]:
test_set

,0
trade_date,
2021-06-21,3.058920
2021-06-22,0.893830
2021-06-23,-2.185780
2021-06-24,-0.004300
2021-06-25,0.002812
...,...
2022-03-11,-0.023977
2022-03-14,-0.019226
2022-03-15,-0.002940


In [43]:
from sklearn.metrics import mean_squared_error
mean_squared_error(list(yuan_test_set['close']),predictions_yuan)


0.00027108880712126067

In [44]:
mean_squared_error(list(test_set.iloc[:,0]),predictions)


0.08263848988636188

In [70]:
yuan_test_set.loc[:,'close']=predictions_yuan
yuan_test_set


C:\Users\tang xiaoqing\AppData\Local\Temp\ipykernel_25316\2457178840.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yuan_test_set.loc[:,'close']=predictions_yuan


,open,high,low,close,amount
trade_date,,,,,
2021-06-21,3.06,3.08,3.06,3.061432,359751.317
2021-06-22,3.07,3.08,3.06,3.071164,285873.151
2021-06-23,3.07,3.08,3.06,3.069248,268733.070
2021-06-24,3.07,3.08,3.06,3.070000,252945.198
2021-06-25,3.08,3.10,3.07,3.070000,390793.156
...,...,...,...,...,...
2022-03-11,3.08,3.11,3.06,3.081495,550569.534
2022-03-14,3.08,3.12,3.07,3.099315,489277.432
2022-03-15,3.09,3.09,3.02,3.088847,886628.898


In [73]:
test_set.iloc[:,0]=predictions
test_set


C:\Users\tang xiaoqing\AppData\Local\Temp\ipykernel_25316\2405024620.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set.iloc[:,0]=predictions


,0
trade_date,
2021-06-21,3.058920
2021-06-22,0.893830
2021-06-23,-2.185780
2021-06-24,-0.004300
2021-06-25,0.002812
...,...
2022-03-11,-0.023977
2022-03-14,-0.019226
2022-03-15,-0.002940


In [74]:

sc = MinMaxScaler(feature_range=(0, 1))
yuan_sc = MinMaxScaler(feature_range=(0, 1))
training_set_scaled = sc.fit_transform(training_set)
testing_set_scaled = sc.fit_transform(test_set)
yuan_training_set_scaled = yuan_sc.fit_transform(yuan_training_set)
yuan_testing_set_scaled = yuan_sc.fit_transform(yuan_test_set)

X_train, y_train = data_split(training_set_scaled, n_timestamp)
yuan_X_train, yuan_y_train = data_split(yuan_training_set_scaled, n_timestamp)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
yuan_X_train = yuan_X_train.reshape(yuan_X_train.shape[0], yuan_X_train.shape[1], 5)

X_test, y_test = data_split(testing_set_scaled, n_timestamp)

X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
yuan_X_test, yuan_y_test = data_split(yuan_testing_set_scaled, n_timestamp)
yuna_X_test = yuan_X_test.reshape(yuan_X_test.shape[0], yuan_X_test.shape[1], 5)


In [75]:
print(X_train.shape)
y_train.shape
print(yuan_X_train.shape)


(3489, 10, 1)
(3489, 10, 5)


In [76]:


model ,yuan_model= cnn_lstm_model(X_train,yuan_X_train)
model.summary() 
yuan_model.summary()
adam = adam_v2.Adam(learning_rate=0.01)
model.compile(optimizer=adam, loss='mse') 
yuan_model.compile(optimizer=adam, loss='mse') 
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=n_epochs,
                    validation_data=(X_test, y_test),
                    validation_freq=1)
yuan_history = yuan_model.fit(yuan_X_train, yuan_y_train,
                              batch_size=32,
                              epochs=n_epochs,
                              validation_data=(yuan_X_test, yuan_y_test),
                              validation_freq=1)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 3489, 16)          32        
_________________________________________________________________
dropout_2 (Dropout)          (None, 3489, 16)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100)               26800     
_________________________________________________________________
flatten_2 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 26,933
Trainable params: 26,933
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_3"
_________________________________________________________________
Layer (type)                

In [77]:
np.mean(history.history['val_loss'])

0.011091978675685822

In [79]:
np.mean(yuan_history.history['val_loss'])

0.08072994038462639

In [127]:
model ,yuan_model= lstm(1,X_train,yuan_X_train)
model.summary() 
yuan_model.summary()
adam = adam_v2.Adam(learning_rate=0.01)
model.compile(optimizer=adam, loss='mse') 
yuan_model.compile(optimizer=adam, loss='mse') 
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=n_epochs,
                    validation_data=(X_test, y_test),
                    validation_freq=1)
yuan_history = yuan_model.fit(yuan_X_train, yuan_y_train,
                              batch_size=32,
                              epochs=n_epochs,
                              validation_data=(yuan_X_test, yuan_y_test),
                              validation_freq=1)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 50)                10400     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 51        
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 50)                11200     
_________________________________________________________________
dense_9 (Dense)              (None, 5)                 255       
Total params: 11,455
Trainable params: 11,455
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
110/110 [====

In [128]:
np.mean(history.history['val_loss'])

0.027992794811725615

In [129]:
np.mean(yuan_history.history['val_loss'])

0.008001245632767677

In [130]:
model ,yuan_model= lstm(3,X_train,yuan_X_train)
model.summary() 
yuan_model.summary()
adam = adam_v2.Adam(learning_rate=0.01)
model.compile(optimizer=adam, loss='mse') 
yuan_model.compile(optimizer=adam, loss='mse') 
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=n_epochs,
                    validation_data=(X_test, y_test),
                    validation_freq=1)
yuan_history = yuan_model.fit(yuan_X_train, yuan_y_train,
                              batch_size=32,
                              epochs=n_epochs,
                              validation_data=(yuan_X_test, yuan_y_test),
                              validation_freq=1)
                            

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_6 (Bidirection (None, 100)               20800     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 101       
Total params: 20,901
Trainable params: 20,901
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_7 (Bidirection (None, 100)               22400     
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 505       
Total params: 22,905
Trainable params: 22,905
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
110/110 [==

In [105]:
np.mean(history.history['val_loss'])


0.2236836740374565

In [104]:
np.mean(yuan_history.history['val_loss'])

0.00850442805327475